In [2]:
import pandas as pd
import yfinance as yf

bist_data = pd.read_csv("/Users/muratisiklan/Desktop/stock-price/Artifacts/bist_data.csv")

In [22]:
from pymongo import MongoClient
import random
import plotly.express as px

def get_closing_prices_from_mongodb(collection, symbol):
    cursor = collection.find({"Symbol": symbol})
    data = list(cursor)
    return data

client = MongoClient('mongodb://127.0.0.1:27017/')
db = client['stockdata']
collection = db['bist']

selected_companies = random.sample(bist_data['Code'].tolist(), 25)

fig = px.line(title='Closing Prices of Randomly Selected Companies', labels={'index': 'Date', 'value': 'Closing Price'})

for company in selected_companies:
    data = get_closing_prices_from_mongodb(collection, company)
    if data:
        dates = [record['Date'] for record in data]
        closing_prices = [record['Close'] for record in data]

        fig.add_scatter(x=dates, y=closing_prices, mode='lines+markers', name=company, line=dict(width=0.2))

fig.update_layout(
    hovermode='x unified', 
    template='plotly',
)

fig.show(renderer="notebook_connected")  
client.close()


In [62]:
def get_stock_data(symbol, start_date, end_date):
    try:
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        return stock_data
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

dat = get_stock_data("BFREN.IS","2022-10-10","2023-12-20")

[*********************100%%**********************]  1 of 1 completed
